# Vietnamese Question Answering with BARTpho-syllable

The question answering we are developing is called **extractive question answering**, which involves posing a question about a document and indentifying answers as spans of text in the document itself. We use the UIT-ViQuAD2.0 dataset, which is the extractive question answering dataset. Other types of question answering includes **generative question answering** which generates answers for open-ended questions.

## Preparing the data

### Download UIT-ViQuAD2.0 dataset

In [1]:
# download datasets module
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("taidng/UIT-ViQuAD2.0")

README.md:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.20M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/735k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28454 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3814 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7301 [00:00<?, ? examples/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 28454
    })
    validation: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 3814
    })
    test: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 7301
    })
})

### Print first element of the training set

In [4]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).
Question:  Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?
Answer:  {'text': ['Lâm Bá Kiệt'], 'answer_start': [507]}


### During training, there is only **one** possible answer. We have to filter datasets which has only **one** possible answer. For validation and test sets, we don't need to filter because we will calculate the score between a predicted answer with all the acceptable answers and take the best score.

In [5]:
raw_datasets['train'] = raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) == 1)

Filter:   0%|          | 0/28454 [00:00<?, ? examples/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 19238
    })
    validation: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 3814
    })
    test: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 7301
    })
})

## Data preprocessing

### Preprocessing the training data

#### Convert the input text to IDs

In [7]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast

model_checkpoint = "vinai/bartpho-syllable"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartphoTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


##### Pass the question and answer to the tokenizer, it will properly insert the special tokens to form the sentence: [CLS] quesion [SEP] context [SEP]

In [8]:
# Try tokenizer with 1 sample
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s> Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?</s></s> Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).</s>'

##### Limit the length to avoid too long contexts. We will dealt with long context by creating several training features from one sample of our dataset, with a sliding window between them

In [9]:
inputs = tokenizer(
    question,
    context,
    max_length=100,   # maximum length of context
    truncation="only_second",   # truncate the context (which is in the second position) when the question with its context is too long
    stride=50,        # the number of overlapping tokens between two successive chunks
    return_overflowing_tokens=True,   # use overflow tokens for the model
    return_offsets_mapping=True,    # map the tokens back to their positions in the context (the index of the token in the context where the answer starts and the index of the token where the answer ends.)
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

<s> Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?</s></s> Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân</s>
<s> Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?</s></s> nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu</s>
<s> Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?</s></s> tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộ

In [10]:
inputs

{'input_ids': [[0, 2854, 603, 247, 11, 957, 351, 760, 203, 111, 26, 50, 641, 168, 358, 82, 65, 232, 1760, 54, 1663, 34, 962, 944, 1276, 286, 2, 2, 957, 351, 760, 3229, 157, 215, 28, 8991, 1492, 1753, 157, 349, 28, 3272, 73, 9, 578, 567, 44, 353, 7, 55, 1709, 1978, 97, 2072, 119, 168, 709, 76, 70, 38, 28, 4596, 24, 59, 1099, 28, 4508, 603, 9, 520, 499, 608, 53, 201, 305, 73, 13, 30, 26, 1009, 1438, 107, 28, 3997, 5, 703, 39, 69, 406, 480, 115, 103, 578, 567, 464, 436, 76, 70, 1394, 2], [0, 2854, 603, 247, 11, 957, 351, 760, 203, 111, 26, 50, 641, 168, 358, 82, 65, 232, 1760, 54, 1663, 34, 962, 944, 1276, 286, 2, 2, 55, 1709, 1978, 97, 2072, 119, 168, 709, 76, 70, 38, 28, 4596, 24, 59, 1099, 28, 4508, 603, 9, 520, 499, 608, 53, 201, 305, 73, 13, 30, 26, 1009, 1438, 107, 28, 3997, 5, 703, 39, 69, 406, 480, 115, 103, 578, 567, 464, 436, 76, 70, 1394, 168, 1709, 1978, 97, 38, 28, 6062, 30, 28, 4596, 5, 451, 9, 208, 578, 567, 76, 70, 34, 208, 867, 2], [0, 2854, 603, 247, 11, 957, 351, 760, 2

In [11]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [12]:
inputs["overflow_to_sample_mapping"]  # divide long contexts into spans of context

[0, 0, 0, 0, 0, 0]

In [13]:
# Try tokenizer with more samples
inputs = tokenizer(
    raw_datasets["train"][2:6]["question"],
    raw_datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

inputs["overflow_to_sample_mapping"]  # divide long contexts into spans of context

<s> Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì?</s></s> Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến</s>
<s> Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì?</s></s> Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch</s>
<s> Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì?</s></s> đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng

[0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3]

##### Converts character-level answer annotations into token-level positions

Since the task of the question answering model is to predict where the answer is located in the context by outputting the start and end token indices.

We will loop through chunks of tokenized context then check if the annotated answer is fully present in each tokenized chunk of the context.
* If the answer is fully within a chunk, it identifies the tokens where the answer starts and ends.
* If the answer is not fully present in a chunk, it assigns `(0, 0)` to indicate no valid answer in that chunk.

We need to find which is the case.


In [14]:
answers = raw_datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
  sample_idx = inputs["overflow_to_sample_mapping"][i]
  answer = answers[sample_idx]
  start_char = answer["answer_start"][0]
  end_char = answer["answer_start"][0] + len(answer["text"][0])
  sequence_ids = inputs.sequence_ids(i)

  # Find the start and end of the context
  idx = 0
  while sequence_ids[idx] != 1:
    idx += 1
  context_start = idx
  while sequence_ids[idx] == 1:
    idx += 1
  context_end = idx - 1

  # If the answer is not fully inside the context, label is (0, 0)
  if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
    start_positions.append(0)
    end_positions.append(0)
  else:
    # Otherwise it's the start and end token positions
    idx = context_start
    while idx <= context_end and offset[idx][0] <= start_char:
      idx += 1
    start_positions.append(idx - 1)
    idx = context_end
    while idx >= context_start and offset[idx][1] >= end_char:
      idx -= 1
    end_positions.append(idx + 1)

start_positions, end_positions

([84, 55, 26, 0, 25, 0, 0, 0, 0, 0, 0, 41, 27, 0, 0],
 [94, 65, 36, 0, 27, 0, 0, 0, 0, 0, 0, 78, 31, 0, 0])

In [15]:
# check if the start and end positions are matched
# For the first feature we find (84, 94) as labels, so let’s compare the theoretical answer with the decoded span of tokens from 84 to 94

idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa, labels give: Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa


In [16]:
# check for index 4, where we set the pair is (0, 0), which means the answer is not in that context chunk
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: Phạm Văn Đồng, decoded example: <s> Chủ tịch Hội đồng Bộ trưởng đầu tiên của nước Cộng hòa xã hội chủ nghĩa Việt Nam là ai?</s></s> Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hò</s>


##### Turn everything into a function to apply on the whole training dataset

In [17]:
max_length = 384
stride = 128

def preprocess_training_dataset(examples):
  questions = [q.strip() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      examples["context"],
      max_length=max_length,
      truncation="only_second",
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length", 
  )

  offset_mapping = inputs.pop("offset_mapping")
  sample_map = inputs.pop("overflow_to_sample_mapping")
  answers = examples["answers"]
  is_impossible = examples["is_impossible"] 
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):
      sample_idx = sample_map[i]

      # Check if the example is impossible and set answers accordingly
      if is_impossible[sample_idx]:
          answer = {"text": examples["plausible_answers"][sample_idx], "answer_start": [0]}
      else:
          answer = answers[sample_idx]

      start_char = answer["answer_start"][0]
      end_char = answer["answer_start"][0] + len(answer["text"][0])
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label is (0, 0)
      if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  return inputs

In [18]:
# apply the function to the whole training dataset
train_dataset = raw_datasets["train"].map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/19238 [00:00<?, ? examples/s]

(19238, 20567)

### Proprocessing the validation data

#### A tiny bit of cleanup of the offset mappings. They will contain offsets for the question and the context, but once we’re in the post-processing stage we won’t have any way to know which part of the input IDs corresponded to the context and which part was the question. So, we’ll set the offsets corresponding to the question to None

In [19]:
def preprocess_validation_dataset(examples):
  questions = [q.strip() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      examples["context"],
      max_length=max_length,
      truncation="only_second",
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  sample_map = inputs.pop("overflow_to_sample_mapping")
  example_ids = []
  is_impossible = examples["is_impossible"]  
  plausible_answers = examples["plausible_answers"]  

  for i in range(len(inputs["input_ids"])):
      sample_idx = sample_map[i]
      example_ids.append(examples["id"][sample_idx])

      # Check if the example is impossible and set answers accordingly
      if is_impossible[sample_idx]:
          answer = {"text": plausible_answers[sample_idx], "answer_start": [0]}
      else:
          answer = examples["answers"][sample_idx]

      sequence_ids = inputs.sequence_ids(i)
      offset = inputs["offset_mapping"][i]
      inputs["offset_mapping"][i] = [
          o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
      ]

  inputs["example_id"] = example_ids
  return inputs

In [20]:
# apply to the whole validation set
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/3814 [00:00<?, ? examples/s]

(3814, 3938)

### Compute the metrics

In [21]:
def compute_metrics(start_logits, end_logits, features, examples):
    import collections
    from tqdm.auto import tqdm
    import numpy as np
    import evaluate

    n_best = 20
    max_answer_length = 200
    metric = evaluate.load("squad")

    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        is_impossible = example.get("is_impossible", False)
        plausible_answers = example.get("plausible_answers", [])

        answers = []
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index].get("offset_mapping", [])

            if not offsets:
                continue

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            # Handle impossible questions by selecting a plausible answer or an empty string
            if is_impossible and plausible_answers:
                predicted_answers.append(
                    {"id": example_id, "prediction_text": plausible_answers[0]["text"]}
                )
            else:
                predicted_answers.append({"id": example_id, "prediction_text": ""})

    # Create references for metric computation
    theoretical_answers = [
        {
            "id": ex["id"],
            "answers": ex["plausible_answers"] if ex.get("is_impossible", False) else ex["answers"],
        }
        for ex in examples
    ]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)


### Train the model

In [22]:
from transformers import AutoModelForQuestionAnswering, AutoModel
model = AutoModelForQuestionAnswering.from_pretrained("vinai/bartpho-syllable")

pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-syllable and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from transformers import Trainer, TrainingArguments
from huggingface_hub import login

epochs = 7  #5, 10
batch_size = 8   
lr = 2e-5

training_args = TrainingArguments(
    output_dir = "checkpoints",
    eval_strategy = "steps", # print evaluation after finishing an epoch
    num_train_epochs=epochs,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=1,
    save_steps=2000,  
    eval_steps=2000,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
    push_to_hub=True,  # automatically prepare for pushing to the Hub
    hub_model_id="REPO_NAME",  # repo name
    hub_token="YOUR_HF_TOKEN",  # your Hugging Face token
    report_to="none", 
)

In [24]:
model.resize_token_embeddings(len(tokenizer))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


MBartScaledWordEmbedding(40031, 1024, padding_idx=1)

In [25]:
import torch
torch.cuda.empty_cache()

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

Step,Training Loss,Validation Loss
2000,1.163000,No log
4000,0.751300,No log
6000,0.414300,No log
8000,0.244900,No log


TrainOutput(global_step=8995, training_loss=0.7871168873454545, metrics={'train_runtime': 22748.8887, 'train_samples_per_second': 6.329, 'train_steps_per_second': 0.395, 'total_flos': 1.1691678870559642e+17, 'train_loss': 0.7871168873454545, 'epoch': 6.994943601711396})

In [27]:
predictions, _, _ = trainer.predict(validation_dataset)


In [28]:

# Unpack predictions based on structure
if isinstance(predictions, tuple):
    start_logits = predictions[0]
    end_logits = predictions[1]
elif isinstance(predictions, dict):
    # Some models may return predictions as a dict
    start_logits = predictions['start_logits']
    end_logits = predictions['end_logits']
else:
    raise ValueError("Unexpected predictions format!")

In [29]:
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/3814 [00:00<?, ?it/s]

{'exact_match': 58.02307288935501, 'f1': 80.1044942573619}

In [30]:
import gc
gc.collect()

31

In [31]:
import torch
torch.cuda.empty_cache()

In [36]:
!pip install transformers huggingface-hub

In [37]:
trainer.push_to_hub(commit_message="Training completed")

CommitInfo(commit_url='https://huggingface.co/lizz4rd/bartpho-syllable-question-answering/commit/d1cfca5ad2ebad6e45ec99816991c47d8f7a79a7', commit_message='Training completed', commit_description='', oid='d1cfca5ad2ebad6e45ec99816991c47d8f7a79a7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lizz4rd/bartpho-syllable-question-answering', endpoint='https://huggingface.co', repo_type='model', repo_id='lizz4rd/bartpho-syllable-question-answering'), pr_revision=None, pr_num=None)